# Исследование алгоритмов для регулярных запросов

## Постановка задачи

Данный эксперимент посвящен исследованию следующих задач достижимости, решаемых в предыдущих работах.

- Достижимость между всеми парами вершин (`tensor_based_rpq`).
- Достижимость для каждой из заданного множества стартовых вершин (`ms_bfs_based_rpq`).

Вопросы, на которые необходимо ответить в ходе исследования.

- Какое представление разреженных матриц и векторов лучше подходит для каждой из решаемых задач?
- Начиная с какого размера стартового множества выгоднее решать задачу для всех пар и выбирать нужные?


## Описание исследуемых решений

Оба решения на вход принимают граф, его множество стартовых и финальных веришни, а также регулярное выражение, которому должны удолетворять пути.


### tensor_based_rpq (Достижимость между всеми парами вершин)

- Из полученного графа и регулярного выражения строим конечные автоматы.
- По полученным автоматам строим их булевы декомпозиции.
- Перемножаем получившиеся булевы декомпозиции с помощью тензорного произведения. 
	- Относительно автоматов этот шаг равносилен их пересечению.
- Объединяем все элементы получившейся булевой декомпозиции.
- Для получившейся матрицы строим транзитивное замыкание.
	- Это транзитивное замыкание отражает все возможные пути в пересечённом автомате.
- По транзитивному замыканию восстанавливаем достижимость вершин в изначальном графе (с учётом ограничения на пути).


### ms_bfs_based_rpq (Достижимость для каждой из заданного множества стартовых вершин)

- Из полученного графа и регулярного выражения строим конечные автоматы.
- По полученным автоматам строим их булевы декомпозиции.
- Создаем булеву декомпозицию для объединения автоматов, вычисляя прямое суммирование булевых декомпозиций предыдущих автоматов.
- Создаём фронт и маску посещенных состояний на основе стартовых состояний.
	- Фронт - это матрица отображающая состояния в которых мы находимся в данный момент.
- Выполняем основной цикл BFS:
    - Перемещаем фронт на основе булевой декомпозиции.
	    - Умножаем фронт на элементы булевой декомпозиции.
	    - Складываем и преобразовываем полившиеся матрицы для корректной дальнейшей работы.
    - Обновляем маску посещенных состояний, добавляя обработанные состояния.
		- А также с помощью маски убираем из фронта уже посещенные состояния.
    - Если маска не изменилась, выходим из цикла.
- По маске посещенных состояний восстанавливаем достижимость вершин в изначальном графе (с учётом ограничения на пути).

## Вспомогательное

In [22]:
import logging
import multiprocessing

# Отключение логирования, чтобы библиотека cfpq_data не отвлекала информационными сообщениями
logging.disable(logging.INFO) 

# Число процессоров для параллельного запуска тестов
CPU_COUNT = min(6, multiprocessing.cpu_count())

## Описание набора данных для экспериментов 

### Графы

В качестве графов для эксперимента были выбраны графы из [предложенного набора](https://formallanguageconstrainedpathquerying.github.io/CFPQ_Data/graphs/index.html). Из них были выбраны графы содержащие меньше 1000 вершин и меньше 1000 рёбер.

In [23]:
GRAPH_NAMES = [
    "skos",
    "wc",
    "generations",
    "travel",
    "univ",
    "atom",
    "biomedical",
    "bzip",
    "foaf",
    "people",
    "pr",
]

import networkx, cfpq_data

def graph_by_name(name: str) -> networkx.MultiDiGraph:
    path = cfpq_data.download(name)
    return cfpq_data.graph_from_csv(path)

GRAPHS = [graph_by_name(name) for name in GRAPH_NAMES]

### Запросы

Для анализа были выбраны следующие шесть регулярных выражений. В каждом из них цифры заменяются на соответствующие метки в соответствии с их популярностью. Отображение цифр на метки происходит следующим образом:

- 1 заменяется на метку, занимающую первое место по распространённости,
- 2 заменяется на метку, занимающую второе место по распространённости,
- и так далее.

Если в графе количество уникальных меток меньше, чем число различных цифр в запросе, то все цифры, для которых не найдено соответствующих меток, заменяются на метку, занимающую последнее место по распространённости

In [24]:
REGEXS = [
    "1*",
    "(1 | 2 | 3 | 4 | 5)*",
    "(1 | 2)*.3",
    "1.1.1.1",
    "1.2.1.2",
    "(1 | 4).(2 | 3)",
]

def spec_regex_for_graph(graph: networkx.MultiDiGraph, regex: str) -> str:
    sorted_labels = cfpq_data.get_sorted_labels(graph)
    for i in range(1, 10):
        if i <= len(sorted_labels):
            new = sorted_labels[i - 1]
        else:
            new = sorted_labels[-1]
        regex = regex.replace(str(i), new)

    return regex

### Стартовые вершины

Стартовые вершины выбираются с помощью [предложенной функции](https://formallanguageconstrainedpathquerying.github.io/CFPQ_Data/reference/graphs/generated/cfpq_data.graphs.utils.multiple_source_utils.html#cfpq_data.graphs.utils.multiple_source_utils.generate_multiple_source). Кроме того для каждого графа генерируется 3 различных по размеру множества стартовых вершин, с размерами равными 0.01, 0.1 и 1 от числа вершин в графе.

In [25]:
seed = 0xbeaf

PARTS = [0.01, 0.1, 1]

In [26]:
from dataclasses import dataclass
from typing import Set, Tuple, List

@dataclass
class TestCase:
    graph_name: str
    graph: networkx.MultiDiGraph
    regex: str
    start_states: Set[int]
    
TEST_CASES_ALGO = []
for graph_ind in range(len(GRAPHS)):
    graph = GRAPHS[graph_ind]
    graph_name = GRAPH_NAMES[graph_ind]
    
    for prt in PARTS:
        statr_states = cfpq_data.generate_multiple_source(graph, int(graph.number_of_nodes()*prt), seed=seed)
        for reg in REGEXS:
            tc = TestCase(graph_name, graph, reg, statr_states)
            TEST_CASES_ALGO.append(tc)

## Описание эксперимента 


### Оборудование

Эксперимент проводился на машине с ОС `Ubuntu MATE 22.04.4 LTS x86_64`, на процессоре `AMD Ryzen 7 4700U`, код запускался на `Python 3.12.5` с зависимостями указанными в этом [файле (../requirements.lock)](../requirements.lock).

### Ход эксперимента

#### Размер стартового множества

Для того чтобы ответить на вопрос "Начиная с какого размера стартового множества выгоднее решать задачу для всех пар и выбирать нужные?" Измерим время работы каждого решения на различных графах, регулярных выражениях и наборов стартовых вершин, а потом сравним полученный результат.
#### Представление матриц

Для ответа на вопрос "Какое представление разреженных матриц и векторов лучше подходит для каждой из решаемых задач?" 
Будут провериться 3 вида матриц:
> 1. [`csc_array`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csc_array.html#scipy.sparse.csc_array "scipy.sparse.csc_array"): Compressed Sparse Column format
> 2. [`csr_array`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_array.html#scipy.sparse.csr_array "scipy.sparse.csr_array"): Compressed Sparse Row format
> 3. [`coo_array`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_array.html#scipy.sparse.coo_array "scipy.sparse.coo_array"): COOrdinate format (aka IJV, triplet format)

Для проведения данного эксперимента будут отобраны несколько наборов графов, регулярных выражений и стартовых вершин на основе результатов предыдущего эксперимента. Основным критерием выбора будет время выполнения алгоритмов на этих наборах, которое должно быть достаточно большим, чтобы минимизировать влияние системных факторов и обеспечить достоверность результатов. Но при этом не чрезмерно длительным, чтобы избежать значительных затрат времени на выполнение эксперимента.

После на этих наборах будут запущены функции решений, в функции `tensor_based_rpq` будут заменены матрицы вида `csc_array`, в функции `ms_bfs_based_rpq` будут заменены матрицы двух видов `csc_array` и `csr_array` (`csr_array` не будет заменяться на `coo_array`, так как последний не поддерживает нужных операций). Для всех изменённых функции будет замерено время работы на каждом из наборов, для дальнейшего сравнения результатов в рамках одного набора.

## Результаты экспериментов 

### Размер стартового множества

In [27]:
from project.graph_util import tensor_based_rpq
from project.bfs_rpq import ms_bfs_based_rpq
import timeit

LAUNCH_COUNT = 12

def _run_rpq_func(rpq_func, regex, graph, start_states):
    def t_func():
        return rpq_func(regex, graph, start_states, set())
    return timeit.timeit(t_func, number=1)

def run_test_case_algo(tc: TestCase) -> Tuple[List[float], List[float]]:
    res_lists = ([], [])
    rpq_funcs = [tensor_based_rpq, ms_bfs_based_rpq]
    with multiprocessing.Pool(processes=CPU_COUNT) as pool:
        for rpq_func, res_lst in zip(rpq_funcs, res_lists):
            results = pool.starmap(_run_rpq_func, [(rpq_func, tc.regex, tc.graph, tc.start_states)] * LAUNCH_COUNT)
            res_lst.extend(results)
    return res_lists


TEST_ALGO_RES = []

for tc in TEST_CASES_ALGO:
    TEST_ALGO_RES.append((tc, run_test_case_algo(tc)))

In [30]:
import numpy
import math
import pandas

OUTLIER_COUNT = int(LAUNCH_COUNT*0.1)

def remove_outlier(lst: List) -> List:
    lst.sort()
    return lst[OUTLIER_COUNT:-OUTLIER_COUNT]

def format_res(lst: List[float]) -> str:
    std = numpy.std(lst)
    mean = numpy.mean(lst)
    dot = max(math.ceil(-math.log10(std)), 0)
    return f"{mean:.{dot}f} ± {std:.{dot}f}"
    

c_graph = []
c_regex = []
c_start_set_size = []
c_tensor_time = []
c_bfs_time = []

for tc, (tensor_t, bfs_t) in TEST_ALGO_RES:
    tensor_t = remove_outlier(tensor_t)
    bfs_t = remove_outlier(bfs_t)
    
    c_graph.append(tc.graph_name)
    c_regex.append(tc.regex)
    c_start_set_size.append(len(tc.start_states))
    c_tensor_time.append(format_res(tensor_t))
    c_bfs_time.append(format_res(bfs_t))


pandas.set_option("display.max_rows", None)
pandas.DataFrame({
    "graph": c_graph,
    "regex": c_regex,
    "size of start state": c_start_set_size,
    "tensor time": c_tensor_time,
    "bfs_time": c_bfs_time
    })



,graph,regex,size of start state,tensor time,bfs_time
0,skos,1*,1,0.013 ± 0.003,0.0182 ± 0.0002
1,skos,(1 | 2 | 3 | 4 | 5)*,1,0.015 ± 0.001,0.0208 ± 0.0009
2,skos,(1 | 2)*.3,1,0.014 ± 0.001,0.0189 ± 0.0006
3,skos,1.1.1.1,1,0.014 ± 0.001,0.0180 ± 0.0002
4,skos,1.2.1.2,1,0.014 ± 0.001,0.0179 ± 0.0003
5,skos,(1 | 4).(2 | 3),1,0.016 ± 0.005,0.0188 ± 0.0003
6,skos,1*,14,0.039 ± 0.002,0.1342 ± 0.0008
7,skos,(1 | 2 | 3 | 4 | 5)*,14,0.041 ± 0.002,0.137 ± 0.001
8,skos,(1 | 2)*.3,14,0.039 ± 0.001,0.137 ± 0.001
9,skos,1.1.1.1,14,0.041 ± 0.003,0.135 ± 0.001


### Представление матриц

#### Описание данных

Для изучения подходящих прелставооений матриц для задачи `tensor_based_rpq`, были выбраны наборы тестов из `TEST_CASES_ALGO` включающие в себя графы:
- `people`
- `pr`

In [43]:
TEST_CASES_MATRIX_TENSOR = []
good_graphs = set(["people", "pr"])


for tc in TEST_CASES_ALGO:
    if(tc.graph_name in good_graphs):
        TEST_CASES_MATRIX_TENSOR.append(tc)

Для изучения подходящих прелставооений матриц для задачи `ms_bfs_based_rpq`, были выбраны наборы тестов из `TEST_CASES_ALGO` включающие в себя графы:
- `people`
- `foaf`

In [44]:
TEST_CASES_MATRIX_BFS = []
good_graphs = set(["people", "foaf"])

for tc in TEST_CASES_ALGO:
    if(tc.graph_name in good_graphs):
        TEST_CASES_MATRIX_BFS.append(tc)

#### Результаты
##### `tensor_based_rpq`

In [86]:
from experiments.help.module_patcher import replace_matrix
from pathlib import Path
from typing import Dict, Callable
import time

MATRIX_TYPES = ["csc", "csr", "coo"]

def create_new_tensor_function() -> Dict[str, Callable]:
    res_dict ={}
    for new in MATRIX_TYPES:
        mod = replace_matrix(Path("../project/graph_util.py"),{"csc": new})
        res_dict[new] = mod.tensor_based_rpq
    return res_dict

def run_test_case_matrix(tc: TestCase, funcs: Dict[str, Callable]) -> Dict[str, List[float]]:
    res_dict ={}
    for matrix_type in funcs.keys():
        rpq_func = funcs[matrix_type]
        time_lst = []
        for _ in range(LAUNCH_COUNT):
            t = _run_rpq_func(rpq_func, tc.regex, tc.graph, tc.start_states)
            time_lst.append(t)
        res_dict[matrix_type] = time_lst
        
    return res_dict


TENSOR_FCNS = create_new_tensor_function()

TEST_MATRIX_TENSOR_RES = []

for tc in TEST_CASES_MATRIX_TENSOR:
    TEST_MATRIX_TENSOR_RES.append((tc, run_test_case_matrix(tc, TENSOR_FCNS)))

In [96]:
c_graph = []
c_regex = []
c_start_set_size = []

c_time_dict: Dict = {}

for tc, (dict_tensor_t) in TEST_MATRIX_TENSOR_RES:
    c_graph.append(tc.graph_name)
    c_regex.append(tc.regex)
    c_start_set_size.append(len(tc.start_states))
   
    for matrix_type in dict_tensor_t.keys():
        lst = c_time_dict.get(matrix_type) 
        if lst is None:
            c_time_dict[matrix_type] = []
            lst = c_time_dict[matrix_type]
        
        all_time_lst = dict_tensor_t[matrix_type]
        all_time_lst = remove_outlier(all_time_lst)
        
        lst.append(format_res(all_time_lst))

pandas.set_option("display.max_rows", None)

pd_dict = {
    "graph": c_graph,
    "regex": c_regex,
    "size of start state": c_start_set_size
    }

pd_dict.update(c_time_dict)

pandas.DataFrame(pd_dict)


,graph,regex,size of start state,csc,csr,coo
0,people,1*,3,0.03074 ± 0.00009,0.0312 ± 0.0002,0.03133 ± 0.00007
1,people,(1 | 2 | 3 | 4 | 5)*,3,0.0336 ± 0.0003,0.0336 ± 0.0004,0.0322 ± 0.0002
2,people,(1 | 2)*.3,3,0.0324 ± 0.0003,0.03241 ± 0.00005,0.0322 ± 0.0003
3,people,1.1.1.1,3,0.0339 ± 0.0004,0.0351 ± 0.0004,0.0332 ± 0.0003
4,people,1.2.1.2,3,0.0345 ± 0.0004,0.0348 ± 0.0003,0.0339 ± 0.0004
5,people,(1 | 4).(2 | 3),3,0.0334 ± 0.0003,0.0338 ± 0.0002,0.0328 ± 0.0002
6,people,1*,33,0.1543 ± 0.0010,0.1553 ± 0.0009,0.156 ± 0.002
7,people,(1 | 2 | 3 | 4 | 5)*,33,0.160 ± 0.003,0.160 ± 0.002,0.1570 ± 0.0005
8,people,(1 | 2)*.3,33,0.1569 ± 0.0004,0.1586 ± 0.0003,0.1581 ± 0.0005
9,people,1.1.1.1,33,0.158 ± 0.002,0.162 ± 0.001,0.161 ± 0.001


##### `ms_bfs_based_rpq`

In [87]:
buf = set(MATRIX_TYPES)
buf.discard("coo")
MATRIX_TYPES_NO_COO = list(buf)


def create_new_bfs_funcs() -> Dict[str, Callable]:
    res_dict ={}
    for new1 in MATRIX_TYPES:
        for new2 in MATRIX_TYPES_NO_COO:
            mod = replace_matrix(Path("../project/bfs_rpq.py"),{"csc": new1, "csr": new2})
            res_dict[f"{new1} | {new2}"] = mod.ms_bfs_based_rpq
    return res_dict

BFS_FNCS = create_new_bfs_funcs()

TEST_MATRIX_BFS_RES = []

for tc in TEST_CASES_MATRIX_BFS:
    TEST_MATRIX_BFS_RES.append((tc, run_test_case_matrix(tc, BFS_FNCS)))

In [101]:
c_graph = []
c_regex = []
c_start_set_size = []
c_time_dict: Dict = {}

for (tc, (dict_bfs_t)) in TEST_MATRIX_BFS_RES:
    c_graph.append(tc.graph_name)
    c_regex.append(tc.regex)
    c_start_set_size.append(len(tc.start_states))
   
    for matrix_type in dict_bfs_t.keys():
        lst = c_time_dict.get(matrix_type) 
        if lst is None:
            c_time_dict[matrix_type] = []
            lst = c_time_dict[matrix_type]
        
        all_time_lst = dict_bfs_t[matrix_type]
        all_time_lst = remove_outlier(all_time_lst)

        lst.append(format_res(all_time_lst))


pandas.set_option("display.max_rows", None)
pd_dict = {
    "graph": c_graph,
    "regex": c_regex,
    "size of start state": c_start_set_size
    }
pd_dict.update(c_time_dict)
pandas.DataFrame(pd_dict)


,graph,regex,size of start state,csc | csc,csc | csr,csr | csc,csr | csr,coo | csc,coo | csr
0,foaf,1*,2,0.0425 ± 0.0004,0.0416 ± 0.0004,0.0429 ± 0.0003,0.0416 ± 0.0003,0.0417 ± 0.0003,0.0408 ± 0.0002
1,foaf,(1 | 2 | 3 | 4 | 5)*,2,0.0440 ± 0.0002,0.0432 ± 0.0004,0.0440 ± 0.0002,0.0441 ± 0.0002,0.0439 ± 0.0004,0.0428 ± 0.0002
2,foaf,(1 | 2)*.3,2,0.0429 ± 0.0002,0.0421 ± 0.0004,0.0430 ± 0.0003,0.0418 ± 0.0003,0.0422 ± 0.0002,0.0415 ± 0.0003
3,foaf,1.1.1.1,2,0.0424 ± 0.0003,0.0414 ± 0.0001,0.0430 ± 0.0003,0.0414 ± 0.0002,0.04162 ± 0.00009,0.0408 ± 0.0001
4,foaf,1.2.1.2,2,0.0423 ± 0.0002,0.0414 ± 0.0001,0.0423 ± 0.0001,0.0414 ± 0.0001,0.0417 ± 0.0002,0.0404 ± 0.0001
5,foaf,(1 | 4).(2 | 3),2,0.04218 ± 0.00006,0.04145 ± 0.00008,0.0423 ± 0.0001,0.0414 ± 0.0001,0.04151 ± 0.00009,0.0411 ± 0.0003
6,foaf,1*,25,0.370 ± 0.002,0.3602 ± 0.0008,0.370 ± 0.003,0.361 ± 0.001,0.370 ± 0.003,0.362 ± 0.001
7,foaf,(1 | 2 | 3 | 4 | 5)*,25,0.3731 ± 0.0009,0.3607 ± 0.0010,0.371 ± 0.003,0.362 ± 0.001,0.370 ± 0.003,0.3615 ± 0.0010
8,foaf,(1 | 2)*.3,25,0.368 ± 0.003,0.360 ± 0.001,0.372 ± 0.001,0.364 ± 0.002,0.3735 ± 0.0008,0.365 ± 0.001
9,foaf,1.1.1.1,25,0.374 ± 0.002,0.372 ± 0.003,0.375 ± 0.001,0.366 ± 0.003,0.372 ± 0.002,0.368 ± 0.003


## Анализ результатов экспериментов 

###  Размер стартового множества

Давайте найдём все случаи, когда выгоднее решать задачу для отдельных пар вершин.

In [102]:
c_graph = []
c_regex = []
c_start_set_size = []
c_tensor_time = []
c_bfs_time = []

for tc, (tensor_t, bfs_t) in TEST_ALGO_RES:
    tensor_t = remove_outlier(tensor_t)
    bfs_t = remove_outlier(bfs_t)
    
    tensor_mean = numpy.mean(tensor_t)
    bfs_mean = numpy.mean(bfs_t)
    
    if(tensor_mean > bfs_mean):
        c_graph.append(tc.graph_name)
        c_regex.append(tc.regex)
        c_start_set_size.append(len(tc.start_states))
        c_tensor_time.append(format_res(tensor_t))
        c_bfs_time.append(format_res(bfs_t))


pandas.set_option("display.max_rows", None)
pandas.DataFrame({
    "graph": c_graph,
    "regex": c_regex,
    "size of start state": c_start_set_size,
    "tensor time": c_tensor_time,
    "bfs_time": c_bfs_time
    })




,graph,regex,size of start state,tensor time,bfs_time


Результаты эксперимента показывают, что для всех исследованных случаев выгоднее запускать задачу для всех пар и затем выбирать нужные.

> Q: Начиная с какого размера стартового множества выгоднее решать задачу для всех пар и выбирать нужные?
>
> A: С любого.

Причина такого "полярного" результата, скорее всего, заключается в неэффективной реализации алгоритма для задачи "достижимости для каждой из заданного множества стартовых вершин". Это подтверждается тем, что в среднем первый алгоритм работает в 3 раз быстрее на выбранных данных.


In [103]:
relative_diff = []

for _, (tensor_t, bfs_t) in TEST_ALGO_RES:
    tensor_t = remove_outlier(tensor_t)
    bfs_t = remove_outlier(bfs_t)
    
    tensor_mean = numpy.mean(tensor_t)
    bfs_mean = numpy.mean(bfs_t)
    
    relative_diff.append(bfs_mean/tensor_mean)
    
print("bfs_time / tensor_time")
print(f"\tMean = {numpy.mean(relative_diff)}")
print(f"\tMin = {min(relative_diff)}")
print(f"\tMax = {max(relative_diff)}")

bfs_time / tensor_time
	Mean = 3.554195807608944
	Min = 1.18143916083217
	Max = 4.9661435607702344


###  Представление матриц

Оценим относительное ускорение при разных видах матриц.
Для `tensor_based_rpq`:

In [113]:
def get_mean_dicts(test_matrix_res: List) -> List[Dict[str, float]]:
    res_lst = []
    for _, (dict_t) in test_matrix_res:
        buf_dict = {}
        for matrix_type in dict_t.keys():
            all_time_lst = dict_t[matrix_type]
            all_time_lst = remove_outlier(all_time_lst)

            buf_dict[matrix_type] = numpy.mean(all_time_lst)
        res_lst.append(buf_dict)
    return res_lst

def get_relative_dicts(mean_dicts: List[Dict], base: str) -> List[Dict]:
    res_lst = []
    for dict_t in mean_dicts:
        buf_dict = {}
        base_seconds = dict_t[base]
        for matrix_type in dict_t.keys():
            seconds = dict_t[matrix_type]
            buf_dict[matrix_type] = seconds/base_seconds
            
        res_lst.append(buf_dict)
    return res_lst


def join_dicts(dicts: List[Dict]) -> Dict:
    main_dict = {}
    for d in dicts:
        for matrix_type in d.keys():
            lst = main_dict.get(matrix_type) 
            if lst is None:
                main_dict[matrix_type] = []
                lst = main_dict[matrix_type]
            val = d[matrix_type]
            lst.append(val)
    return main_dict

def get_relative_dict(test_matrix_res: List[Dict], base: str) -> Dict:
    return join_dicts(get_relative_dicts(get_mean_dicts(test_matrix_res), base))

def print_statistic_for_relative_dict(rdict: Dict):
    for name in rdict:
        lst = rdict[name]
        min_v = min(lst)*100
        max_v = max(lst)*100        
        mean_v = numpy.mean(lst)*100      
        
        print(f"<{name}>: mean = {mean_v:6.1f}% | min = {min_v:6.1f}% | max = {max_v:6.1f}% |")

tensor_rd = get_relative_dict(TEST_MATRIX_TENSOR_RES, "csc")
print_statistic_for_relative_dict(tensor_rd)

<csc>: mean =  100.0% | min =  100.0% | max =  100.0% |
<csr>: mean =  101.2% | min =   99.3% | max =  104.6% |
<coo>: mean =  100.7% | min =   96.0% | max =  102.9% |


Для `ms_bfs_based_rpq`:

In [114]:
bfs_rd = get_relative_dict(TEST_MATRIX_BFS_RES, "csc | csr")
print_statistic_for_relative_dict(bfs_rd)

<csc | csc>: mean =  101.7% | min =   99.6% | max =  103.4% |
<csc | csr>: mean =  100.0% | min =  100.0% | max =  100.0% |
<csr | csc>: mean =  101.9% | min =   99.8% | max =  103.8% |
<csr | csr>: mean =  100.0% | min =   98.1% | max =  102.2% |
<coo | csc>: mean =  101.3% | min =   98.6% | max =  103.6% |
<coo | csr>: mean =   99.4% | min =   96.5% | max =  101.2% |


Анализ результатов показал, что изменение представления разреженных матриц не оказало значительного влияния на время работы алгоритмов. Во всех случаях среднее изменение времени работы составило менее 2%, что указывает на то, что для написанных реализаций задач не имеет разницы какой вид разреженных матриц использовать.

> Q: Какое представление разреженных матриц и векторов лучше подходит для каждой из решаемых задач?
>
> A: Все три представления матриц (CSR, CSC и COO) показали примерно одинаковую производительность на выбранных данных. Следовательно, любой из этих вариантов может быть использован для решения задачи.

Наблюдаемое поведение можно объяснить тем, что все представления матриц умножаются примерно за одинаковое время в реализации tensor_based_rpq. Более интересным является тот факт, что реализация ms_bfs_based_rpq также не показала значительного изменения скорости, несмотря на то, что там перемножаются матрицы разных типов. Это, скорее всего, связано с наличием тяжёлых операций в алгоритме для CSR-матриц, таких как операция разрезания строк (row slicing operation), которая сводит на нет преимущества быстрого умножения CSR на CSC.